In [1]:
!pip install pydicom numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.0 MB/s eta 0:00:00


In [1]:
import pydicom
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import ImplicitVRLittleEndian
import numpy as np
import os

# --- Part 1: Create a simple DICOM file ---

# Create a temporary file name
filename = 'dicom_test.dcm'

# Define the file meta information
file_meta = Dataset()
file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'  # CT Image Storage
file_meta.MediaStorageSOPInstanceUID = '1.2.3'
file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
file_meta.ImplementationClassUID = '1.2.3.4.5.6.7.8.9.1'

# Create the DICOM dataset
ds = FileDataset(filename, {}, file_meta=file_meta, preamble=b'\0' * 128)

# Add key DICOM attributes (metadata)
ds.PatientName = "Doe^John"
ds.PatientID = "12345"
ds.StudyDate = "20250917"
ds.Modality = "CT"
ds.SeriesInstanceUID = "1.2.3.4"
ds.SOPInstanceUID = "1.2.3.4.5"
ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.2'

# Define the pixel data (a simple 64x64 grayscale image)
# Create a NumPy array with a simple gradient
pixel_data = np.zeros((64, 64), dtype=np.uint16)
for i in range(64):
    for j in range(64):
        pixel_data[i, j] = i * 4

ds.Rows = pixel_data.shape[0]
ds.Columns = pixel_data.shape[1]
ds.SamplesPerPixel = 1
ds.PhotometricInterpretation = "MONOCHROME2"
ds.BitsAllocated = 16
ds.BitsStored = 16
ds.HighBit = 15
ds.PixelRepresentation = 0  # 0 for unsigned integer

# Set the pixel data
ds.PixelData = pixel_data.tobytes()

print(f"--- Creating a new DICOM file: {filename} ---")
ds.save_as(filename)
print("File created successfully.\n")


# --- Part 2: Read and inspect the DICOM file ---

# Read the DICOM file
try:
    ds_read = pydicom.dcmread(filename)
    print(f"--- Reading DICOM file: {filename} ---")
    print("Metadata:")
    
    # Access and print some key attributes
    print(f"  Patient Name: {ds_read.PatientName}")
    print(f"  Patient ID: {ds_read.PatientID}")
    print(f"  Modality: {ds_read.Modality}")
    print(f"  Study Date: {ds_read.StudyDate}")
    
    # Access pixel data and print its properties
    print("\nPixel Data:")
    pixel_array = ds_read.pixel_array
    print(f"  Pixel array shape: {pixel_array.shape}")
    print(f"  Pixel array data type: {pixel_array.dtype}")
    print("  Pixel data has been successfully loaded.\n")
    
    # Clean up the created file
    #os.remove(filename)
    #print(f"Demo complete. File '{filename}' deleted.")

except FileNotFoundError:
    print(f"Error: The file '{filename}' was not found. Please ensure it was created correctly.")


--- Creating a new DICOM file: dicom_test.dcm ---
File created successfully.

--- Reading DICOM file: dicom_test.dcm ---
Metadata:
  Patient Name: Doe^John
  Patient ID: 12345
  Modality: CT
  Study Date: 20250917

Pixel Data:
  Pixel array shape: (64, 64)
  Pixel array data type: uint16
  Pixel data has been successfully loaded.

